CIS 9 - Final Project NLP

1.Name: Alyssa Ung

Mental health is a very important topic to me and with the pandemic keeping everyone away from their loved ones, it can be hard to reach out and talk to people. I decided to use datasets from reddit because the data allows users to type more that 250 characters which will allow for more words to be analyzed. 

For this project, I aim to train a model that will detect if an input text is a text containing depressive/suicidal context or a regular text that seems harmless. I believe that this can be the start to making social media safer by preventing self harm and implementing preventive measures for those at risk for self harm.

I was insipred by Samuel He's project: (https://github.com/hesamuel/goodbye_world). In his project he analyzed the difference between posts that were tagged as depression vs posts that were of suicide ideation. 

For my input data, I randomly grabbed 2500 reddit posts from each of the datasets below. One dataset is a combination of depressive/suicidal texts and the other is non-depressive/sucidal text. Non-depressive/suicidal text have the label 0 while depressice/suicidal text has the label 1.


2. Datasets from:

https://www.kaggle.com/nikhileswarkomati/suicide-watch

https://www.kaggle.com/mswarbrickjones/reddit-selfposts?select=rspct.tsv


In [172]:
# import statements
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import contractions

3. Here we read in the dataframe for the non-depressive/suicidal text, randomly sample 2500 texts, and label it 0. The shape and some data is printed to show the dataframe.

In [133]:
input1 = pd.read_csv("rspct.csv")

newdf1 = input1[['selftext']]
newdf1 = newdf1.rename(columns = {'selftext': 'text'})
newdf1['label'] = 0

df1 = newdf1.sample(n = 2500)
print(df1.shape)
print(df1.head())


(2500, 2)
                                                     text  label
102386  After a season on FM18 where a variety of fact...      0
586376  We inherited a coin collection and are having ...      0
848382  https://imgur.com/aFnpPdj<lb><lb>I'm not the f...      0
441363  I'm in the position where I've got two XDJ 100...      0
9182    A few days ago I saw a post here on reddit nam...      0


Here we read in the dataframe for the depressive/suicidal text, randomly sample 2500 texts, and label it 1. The shape and some data is printed to show the dataframe.

In [134]:
input2 = pd.read_csv("Cleaned_Depression_Vs_Suicide.csv")
newdf2 = input2.dropna()

df2 = newdf2.sample(n = 2500)
# (df2.groupby('class').size()/df2['class'].count())*100
df2.drop(columns=['class'], inplace=True)
df2['label'] = 1

print(df2.shape)
print(df2.head())


/Users/alyssaung/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(2500, 2)
                                                     text  label
418927  TraumaI haven't been to school in 2 weeks. I c...      1
404108  finally feel happyNow that I've made a foolpro...      1
5510    Reverse Midas Touch...Instead of gold, everyth...      1
149884  She's... strange.Today I went on a "date"(I've...      1
262119  anti anxiety meds recommendations?I've been ta...      1


Both dataframes are then combined and shuffled so that the data will be random

In [135]:
df = pd.concat([df1,df2], axis=0)
df = df.sample(frac=1)

print(df.head())

                                                     text  label
507416  I arrived at the contest excited to compete fo...      0
272263  So, I do work with the Wounded Warriors, where...      0
113461  My friend’s threatening to commit suicide and ...      1
466735  idfkim really tired of trying and tired of the...      1
86922   I am done.It is straining me so much. I can't ...      1


In this cell, the y variable and the X variable is created from the dataframe. The index of the X variable is has been reset to allow for easier preprocessing of the data during tokenization

In [269]:
y = df.label
X = df.drop(columns=['label'])
X.reset_index(drop=True, inplace=True)

X.head()

,text
0,I arrived at the contest excited to compete fo...
1,"So, I do work with the Wounded Warriors, where..."
2,My friend’s threatening to commit suicide and ...
3,idfkim really tired of trying and tired of the...
4,I am done.It is straining me so much. I can't ...


In this cell, I removed some spam from one of the datasets (<lb>) and also links.

In [225]:
X = X.replace(to_replace ='<lb>', value = '', regex = True)
X = X.replace(to_replace ='(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', value = '', regex = True)

I extended the list of stop words, thanks to this thread on github https://gist.github.com/sebleier/554280. This would allow me to further narrow down common words. I also added in a few myself after running it through the model and checking the top words.

In [249]:
stop_words_extended = {"a", "about", "above", "after", "again", "against", "ago", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "ive", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would", "able", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "afterwards", "ah", "almost", "alone", "along", "already", "also", "although", "always", "among", "amongst", "announce", "another", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "arent", "arise", "around", "aside", "ask", "asking", "auth", "available", "away", "awfully", "b", "back", "became", "become", "becomes", "becoming", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "believe", "beside", "besides", "beyond", "biol", "brief", "briefly", "c", "ca", "came", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "couldnt", "date", "different", "done", "downwards", "due", "e", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "former", "formerly", "forth", "found", "four", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "happens", "hardly", "hed", "hence", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hi", "hid", "hither", "home", "howbeit", "however", "hundred", "id", "ie", "im", "immediate", "immediately", "importance", "important", "inc", "indeed", "index", "information", "instead", "invention", "inward", "itd", "it'll", "j", "k", "keep", "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "made", "mainly", "make", "makes", "many", "may", "maybe", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "moreover", "mostly", "mr", "mrs", "much", "mug", "must", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "nobody", "non", "none", "nonetheless", "noone", "normally", "nos", "noted", "nothing", "nowhere", "obtain", "obtained", "obviously", "often", "oh", "ok", "okay", "old", "omitted", "one", "ones", "onto", "ord", "others", "otherwise", "outside", "overall", "owing", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "said", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "shed", "shes", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "sufficiently", "suggest", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th", "thank", "thanks", "thanx", "thats", "that've", "thence", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "theyd", "theyre", "thing","think", "thou", "though", "thoughh", "thousand", "throug", "throughout", "thru", "thus", "til", "time", "tip", "together", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "unfortunately", "unless", "unlike", "unlikely", "unto", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "wasnt", "way", "wed", "welcome", "went", "werent", "whatever", "what'll", "whats", "whence", "whenever", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "whim", "whither", "whod", "whoever", "whole", "who'll", "whomever", "whos", "whose", "widely", "willing", "wish", "within", "without", "wont", "words", "world", "wouldnt", "www", "x", "yes", "yet", "youd", "youre", "z", "zero", "a's", "ain't", "allow", "allows", "apart", "appear", "appreciate", "appropriate", "associated", "best", "better", "c'mon", "c's", "cant", "changes", "clearly", "concerning", "consequently", "consider", "considering", "corresponding", "course", "currently", "definitely", "described", "despite", "entirely", "exactly", "example", "going", "greetings", "hello", "help", "hopefully", "ignored", "inasmuch", "indicate", "indicated", "indicates", "inner", "insofar", "it'd", "keep", "keeps", "novel", "presumably", "reasonably", "second", "secondly", "sensible", "serious", "seriously", "sure", "t's", "third", "thorough", "thoroughly", "three", "well", "wonder", "a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "co", "op", "research-articl", "pagecount", "cit", "ibid", "les", "le", "au", "que", "est", "pas", "vol", "el", "los", "pp", "u201d", "well-b", "http", "volumtype", "par", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a1", "a2", "a3", "a4", "ab", "ac", "ad", "ae", "af", "ag", "aj", "al", "an", "ao", "ap", "ar", "av", "aw", "ax", "ay", "az", "b1", "b2", "b3", "ba", "bc", "bd", "be", "bi", "bj", "bk", "bl", "bn", "bp", "br", "bs", "bt", "bu", "bx", "c1", "c2", "c3", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "cl", "cm", "cn", "cp", "cq", "cr", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d2", "da", "dc", "dd", "de", "df", "di", "dj", "dk", "dl", "do", "dp", "dr", "ds", "dt", "du", "dx", "dy", "e2", "e3", "ea", "ec", "ed", "ee", "ef", "ei", "ej", "el", "em", "en", "eo", "ep", "eq", "er", "es", "et", "eu", "ev", "ex", "ey", "f2", "fa", "fc", "ff", "fi", "fj", "fl", "fn", "fo", "fr", "fs", "ft", "fu", "fy", "ga", "ge", "gi", "gj", "gl", "go", "gr", "gs", "gy", "h2", "h3", "hh", "hi", "hj", "ho", "hr", "hs", "hu", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ic", "ie", "ig", "ih", "ii", "ij", "il", "in", "io", "ip", "iq", "ir", "iv", "ix", "iy", "iz", "jj", "jr", "js", "jt", "ju", "ke", "kg", "kj", "km", "ko", "l2", "la", "lb", "lc", "lf", "lj", "ln", "lo", "lr", "ls", "lt", "m2", "ml", "mn", "mo", "ms", "mt", "mu", "n2", "nc", "nd", "ne", "ng", "ni", "nj", "nl", "nn", "nr", "ns", "nt", "ny", "oa", "ob", "oc", "od", "of", "og", "oi", "oj", "ol", "om", "on", "oo", "oq", "or", "os", "ot", "ou", "ow", "ox", "oz", "p1", "p2", "p3", "pc", "pd", "pe", "pf", "ph", "pi", "pj", "pk", "pl", "pm", "pn", "po", "pq", "pr", "ps", "pt", "pu", "py", "qj", "qu", "r2", "ra", "rc", "rd", "rf", "rh", "ri", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "rv", "ry", "s2", "sa", "sc", "sd", "se", "sf", "si", "sj", "sl", "sm", "sn", "sp", "sq", "sr", "ss", "st", "sy", "sz", "t1", "t2", "t3", "tb", "tc", "td", "te", "tf", "th", "ti", "tj", "tl", "tm", "tn", "tp", "tq", "tr", "ts", "tt", "tv", "tx", "ue", "ui", "uj", "uk", "um", "un", "uo", "ur", "ut", "va", "wa", "vd", "wi", "vj", "vo", "wo", "vq", "vt", "vu", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y2", "yj", "yl", "yr", "ys", "yt", "zi", "zz"}

This preprocess method removes all numbers and punctuations, lowercase all words, remove all stop words and stem all the words. This will allow the count vectorizer to note the frequency of each word better.

In [250]:
tokenizer = RegexpTokenizer('[a-zA-Z]+')
stemmer = PorterStemmer()
stop_words=set(stopwords.words("english"))

def preprocess(s) :
    w = tokenizer.tokenize(s.lower())
    w = [word for word in w if word not in stop_words]
    w = [word for word in w if word not in stop_words_extended]
    w = [stemmer.stem(word) for word in w]
    return ' '.join(w)

X_processed = pd.Series([preprocess(X.loc[i,'text']) for i in range(len(X))]).to_frame()
X_processed.head()

,0
0,arriv contest excit compet year friend regist ...
1,work wound warrior spend week hotel feed meal ...
2,friend threaten commit suicid helpmi friend ex...
3,idfkim tire tire pain live life pointless wort...
4,strain fall asleep night stress school year we...


The count vectorizer counts the frequency of each word prepares the data to be used by the model.

In [251]:
vect = CountVectorizer()
vect.fit(X_processed[0])
X_vectors = vect.transform(X_processed[0])

The data is then split into training sets and testing sets

In [265]:
X_train, X_test, y_train, y_test = train_test_split(X_vectors,y,test_size=0.2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4000, 19321) (4000,) (1000, 19321) (1000,)


Here we use the Multinomial Naive Bayes classifier as it has a higher success rate compared to other algorithms when used in text classification, spam filtering and sentiment analysis.

In [266]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

Conclusion:

Our accuracy turns out to be around 92% accurate. Accuracy is used here because the distribution of data is similar and balanced. Our confusion matrix shows a low error rate.
This looks pretty good and this model can be used to sweep through a feed with random text and determine if action should be taken for preventive measures.

In [267]:
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred, labels=[0,1]))

0.925
[[446  65]
 [ 10 479]]


4. I wanted to analyze further the frequency of words present in the datasets for depressive/suicidal vs non-depressive/suicidal. So I preprocess the dataframes to extract the relevant information

In [255]:
X_non = df1.drop(columns=['label']) #non-depressive/suicidal
X_is = df2.drop(columns=['label']) #depressive/suicidal
X_non.reset_index(drop=True, inplace=True)
X_is.reset_index(drop=True, inplace=True)

In [270]:
X_is = X_is.replace(to_replace ='<lb>', value = '', regex = True)
X_is = X_is.replace(to_replace ='(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', value = '', regex = True)

X_nonpp = pd.Series([preprocess(X_non.loc[i,'text']) for i in range(len(X_non))]).to_frame()
X_ispp = pd.Series([preprocess(X_is.loc[i,'text']) for i in range(len(X_is))]).to_frame()


In [271]:
# Top words for text with non-depressive/suicidal content

vect.fit(X_nonpp[0])
Xnon_vectors = vect.transform(X_nonpp[0])
sum_words = Xnon_vectors.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

print(words_freq[:50])

[('work', 816), ('feel', 668), ('year', 604), ('start', 598), ('tab', 580), ('peopl', 526), ('day', 524), ('good', 506), ('play', 454), ('game', 408), ('guy', 367), ('question', 350), ('post', 345), ('thought', 340), ('love', 331), ('week', 330), ('thing', 328), ('month', 314), ('lot', 311), ('read', 304), ('set', 288), ('gt', 287), ('bit', 276), ('pretti', 274), ('point', 273), ('long', 254), ('issu', 251), ('experi', 249), ('want', 239), ('idea', 235), ('problem', 235), ('happen', 228), ('great', 223), ('edit', 217), ('life', 215), ('person', 214), ('place', 212), ('test', 211), ('chang', 209), ('watch', 206), ('hope', 201), ('talk', 200), ('wonder', 193), ('car', 193), ('friend', 192), ('hour', 188), ('think', 188), ('interest', 187), ('card', 183), ('amp', 182)]


In [258]:
# Top words for text with depressive/suicidal content

vect.fit(X_ispp[0])
Xis_vectors = vect.transform(X_ispp[0])
sum_word = Xis_vectors.sum(axis=0)
word_freq = [(word, sum_word[0, idx]) for word, idx in vect.vocabulary_.items()]
word_freq =sorted(word_freq, key = lambda x: x[1], reverse=True)

print(word_freq[:50])

[('feel', 3842), ('life', 2303), ('year', 1938), ('depress', 1868), ('friend', 1835), ('peopl', 1595), ('day', 1561), ('live', 1179), ('work', 1146), ('fuck', 1134), ('talk', 1132), ('thing', 1058), ('start', 980), ('thought', 973), ('school', 910), ('love', 873), ('suicid', 868), ('kill', 846), ('famili', 812), ('good', 801), ('happi', 740), ('hate', 709), ('care', 708), ('job', 703), ('die', 701), ('month', 665), ('person', 660), ('week', 586), ('bad', 584), ('told', 562), ('point', 553), ('parent', 545), ('long', 539), ('shit', 509), ('want', 504), ('lot', 498), ('felt', 479), ('hard', 470), ('reason', 452), ('hurt', 433), ('pain', 430), ('mom', 429), ('post', 416), ('sad', 415), ('wors', 411), ('hope', 408), ('problem', 405), ('think', 401), ('happen', 389), ('leav', 381)]


From the analysis above, we are able to see some those words frequently used by someone who is depressed/suicidal. This shows how our model using the multinomial naive bayes benefits from the word counts in the text and can be considered a reliable model. We can use the patterns to help us analyze the patterns to further improve the model. We can also try to approach it at a different angle and analyze more than just the texts and their intention. Given more time for research, I would like to also have the model differenciate a text that offers support for depression/sharing of experience vs suicidal ideation or even a newspiece reporting on an incident vs actual suicidal ideation/attempt.

Samuel He's project titled "Goodbye World" already shared that there was a significant correlation between gender and suicide. It would also be really cool to work on that further and find out if the patterns in the text reveal the cause for that and if the causes were similar for the genders. 
